<a href="https://colab.research.google.com/github/a-trefilova/diploma/blob/main/diploma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from simpletransformers.classification import ClassificationModel
import logging
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
from nltk.corpus import stopwords
import os

# Настройка логирования
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Загрузка стоп-слов (для метода очистки)
stop_words = set(stopwords.words('russian'))

# Метод очистки текста от лишних символов
def remove_unneeded_symbols(text, stop_words):
    import re
    import string

    # Очистка резюме от ссылок http или https
    text = re.sub('http[s]?\\S+\\s*', ' ', text)

    # Очистка резюме от пунктуационных знаков
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Определение имен месяцев в русском языке
    months = ["Январь", "Янв", "Февраль", "Фев", "Март", "Мар", "Апрель", "Апр",
              "Май", "Июнь", "Июн", "Июль", "Июл", "Август", "Авг",
              "Сентябрь", "Сен", "Октябрь", "Окт", "Ноябрь", "Ноя", "Декабрь", "Дек"]

    # Регулярное выражение для удаления месяцев
    months_regex = r'\b(' + '|'.join(months) + r')\b'
    text = re.sub(months_regex, '', text, flags=re.IGNORECASE)

    # Регулярное выражение для удаления всех цифр
    digits_regex = r'\d+'
    text = re.sub(digits_regex, '', text)

    # Удаление всех множественных пробелов и стоп-слов
    text = ' '.join([word for word in text.split() if word.lower() not in stop_words])

    return text.strip().lower()

# Загрузка данных с резюме
data = pd.read_csv('Resume_translated_final.csv')

# Предобработка данных — очистка текста и лишних символов
data['cleaned_resume'] = data['Resume_str'].apply(lambda x: remove_unneeded_symbols(x, stop_words))

# Преобразование категорий в числовые метки
data['labels'] = data['Category'].astype('category').cat.codes

# Создание DataFrame с текстом и метками
df = pd.DataFrame({
    'text': data['cleaned_resume'],
    'labels': data['labels']
})

# Разделение данных на обучающую и тестовую выборки
train_df, eval_df = train_test_split(df, test_size=0.2, random_state=42)

# Создаем модель классификации на базе DistilBERT
model = ClassificationModel(
    "distilbert",
    "distilbert-base-multilingual-cased",
    num_labels=len(df['labels'].unique()),
    use_cuda=False,
    args={
        'train_batch_size': 4,  # Меньший размер батча для ускорения
        'eval_batch_size': 4,
        'num_train_epochs': 5,  # Увеличиваем количество эпох
        'max_seq_length': 64,  # Уменьшаем длину последовательностей для ускорения
        'learning_rate': 4e-5,
        'save_eval_checkpoints': False,
        'save_model_every_epoch': False,
        'overwrite_output_dir': True,
        'class_weights': "balanced"  # Взвешенные классы для борьбы с дисбалансом
    }
)

# Тренируем модель на обучающих данных
model.train_model(train_df)

# Оцениваем модель на тестовой выборке
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

# Получаем истинные метки и предсказания
true_labels = eval_df['labels'].tolist()
predicted_labels, _ = model.predict(eval_df['text'].tolist())

# Используем labels и target_names для генерации итогового отчета о классификации
target_names = data['Category'].astype('category').cat.categories
unique_labels = np.unique(true_labels)

# Генерация итогового отчета о классификации с учетом существующих меток
final_report = classification_report(true_labels, predicted_labels, labels=unique_labels, target_names=target_names[:len(unique_labels)])

# Вывод итогового отчета о классификации
print("\nИтоговый отчет о классификации:\n")
print(final_report)

# Сохранение обученной модели
output_dir = 'outputs_model'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

model.save_model(output_dir)

print(f"Модель сохранена в директорию: {output_dir}")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/497 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/497 [00:00<?, ?it/s]

...

Running Evaluation:   0%|          | 0/125 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/125 [00:00<?, ?it/s]


Итоговый отчет о классификации:

                           precision    recall  f1-score   support

                       HR       0.82      1.00      0.90        18
                  АВИАЦИЯ       0.45      0.71      0.56        21
               АВТОМОБИЛЬ       0.00      0.00      0.00         6
                  АДВОКАТ       0.81      0.73      0.77        30
          БАНКОВСКОЕ ДЕЛО       0.52      0.52      0.52        23
       БИЗНЕС-ДЕВЕЛОПМЕНТ       0.00      0.00      0.00         2
                      БПО       1.00      0.97      0.98        29
                БУХГАЛТЕР       0.95      1.00      0.97        19
                 ДИЗАЙНЕР       0.65      0.65      0.65        20
          ЗДРАВООХРАНЕНИЕ       0.00      0.00      0.00         1
                ИНЖЕНЕРИЯ       0.79      0.95      0.86        20
               ИНЖИНИРИНГ       0.92      0.85      0.88        26
ИНФОРМАЦИОННЫЕ ТЕХНОЛОГИИ       0.50      0.47      0.48        17
                ИСКУССТВА  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [6]:
 !pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 16.2 MB/s eta 0:00:00
   ━━━